## **CNN Analysis for fake news detection with random initialization**

 **Load the required libraries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import nltk

nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
stopwords_en = stopwords.words("english")

import re

# Importing required libraries
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords

# keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from keras.layers import Embedding

from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from keras.layers import Activation, Flatten, Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
import keras
from keras.layers.core import Reshape, Flatten
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam
from keras.models import Model
from keras import regularizers

# gensim
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


**Data loading and splitting**

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
    # Load the train dataset
cod_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/fake-news/train.csv")
    # split for cross-validation (train-64%, validation 16% and test 20%)
cod_train2, test= train_test_split(cod_train, random_state=0, test_size=0.2)
train, val =  train_test_split(cod_train2, random_state = 0,test_size=0.2)

**Tokenization**

In [ ]:
max_features = 100000 # max num words
maxlen = 250 
embedding_size = 200

# create the tokenizer with the maximum number of words to keep, 
# based on word frequency. 
# Only the most common num_words-1 words will be kept.
tokenizer = Tokenizer(num_words=max_features, oov_token = True)

train['text']=train['text'].astype(str)
test['text']=test['text'].astype(str)
val['text']=val['text'].astype(str)

# fit the tokenizer on the texts
tokenizer.fit_on_texts(list(train['text']))

# Transforms each text in texts to a sequence of integers.
train_X = tokenizer.texts_to_sequences(train['text'])
test_X = tokenizer.texts_to_sequences(test['text'])
val_X = tokenizer.texts_to_sequences(val['text'])

# transforms a list of num_samples sequences (lists of integers)
# into a 2D Numpy array of shape (num_samples, num_timesteps).
train_X = pad_sequences(train_X, maxlen = maxlen)
test_X = pad_sequences(test_X, maxlen = maxlen)
val_X = pad_sequences(val_X, maxlen = maxlen)

train_y = train['label']
test_y = test['label']
val_y = val['label']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

**Constructing the CNN with random initialization**

It is composed of 2 convolutional networks of 1 dimension, a max pooling, and a global max pooling layer.

- Activation Function: ReLU has been used as the activation function. It is a non-linear activation function, helping complex relationships in the data to be captured by the model.

- Optimizer: Adam optimizer, an adaptive learning rate optimizer.

- Loss function: The network will be trained to output a probability over the 2 classes using Sigmoid Loss.

In [ ]:
#CONV1D 
num_filters = 200
sequence_length = train_X.shape[1]
weight_decay = 1e-4


model = Sequential()
model.add(Embedding(max_features, embedding_size, input_length = sequence_length))
model.add(Conv1D(num_filters, 7, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Conv1D(num_filters, 7, activation='relu', padding='same'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Dense(1, activation='sigmoid'))  #multi-label (k-hot encoding)

In [ ]:
val_y22 = np.array(val_y, dtype=np.float)
val_x22 = np.array(val_X, dtype=np.float)
test_y22 = np.array(test_y, dtype=np.float)
test_x22 = np.array(test_X, dtype=np.float)
train_y22 = np.array(train_y, dtype=np.float)
train_x22 = np.array(train_X, dtype=np.float)


and compiling it

In [ ]:
#FOR CONV1D 

# Compiling Model using optimizer
opt = Adam(lr=1e-3)
model.compile(loss='binary_crossentropy',optimizer=opt, metrics=['accuracy',keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.TruePositives()])
#model.summary()

# Fitting Model to the data
# Save the model after every epoch.
saveBestModel = keras.callbacks.ModelCheckpoint('/content/drive/My Drive/TFMColab/best_model.hdf5', monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
# Stop training when a monitored quantity has stopped improving.
earlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
batch_size = 200
epochs = 5

model.fit(train_X, train_y, batch_size=batch_size, epochs=epochs, validation_data=(val_X, val_y), callbacks=[saveBestModel, earlyStopping])


Epoch 1/5
67/67 [==============================] - 52s 271ms/step - loss: 0.5772 - accuracy: 0.6683 - precision: 0.7024 - recall: 0.5995 - true_positives: 2119.1765 - val_loss: 0.1920 - val_accuracy: 0.9237 - val_precision: 0.9380 - val_recall: 0.9076 - val_true_positives: 1512.0000
Epoch 2/5
67/67 [==============================] - 17s 254ms/step - loss: 0.1182 - accuracy: 0.9595 - precision: 0.9621 - recall: 0.9549 - true_positives: 3257.6912 - val_loss: 0.1562 - val_accuracy: 0.9417 - val_precision: 0.9688 - val_recall: 0.9130 - val_true_positives: 1521.0000
Epoch 3/5
67/67 [==============================] - 17s 258ms/step - loss: 0.0299 - accuracy: 0.9935 - precision: 0.9943 - recall: 0.9928 - true_positives: 3421.3824 - val_loss: 0.2072 - val_accuracy: 0.9357 - val_precision: 0.9226 - val_recall: 0.9514 - val_true_positives: 1585.0000
Epoch 4/5
67/67 [==============================] - 17s 254ms/step - loss: 0.0107 - accuracy: 0.9981 - precision: 0.9979 - recall: 0.9982 - true_posi

Then the **metrics** are computed

In [ ]:
loss, accuracy, precision, recall, true_positives = model.evaluate(test_X, test_y, batch_size=batch_size)


21/21 [==============================] - 1s 28ms/step - loss: 0.2532 - accuracy: 0.9406 - precision: 0.9456 - recall: 0.9371 - true_positives: 1981.0000


In [ ]:
mult_pr=precision*recall
sum_pr=precision+recall
div=mult_pr/sum_pr
f1_score=2*div

In [ ]:
print('Loss:',loss)
print('Accuracy:',accuracy)
print('Precision:',precision)
print('Recall:',recall)
print('f1 score:',f1_score)
print('True positives:',true_positives)

Loss: 0.2532398998737335
Accuracy: 0.940625011920929
Precision: 0.9455847144126892
Recall: 0.9370861053466797
f1 score: 0.9413162279931595
True positives: 1981.0


In [ ]:
pred_y = model.predict(test_X, batch_size=batch_size)
pred_y


array([[7.2118390e-05],
       [9.9998295e-01],
       [1.0000000e+00],
       ...,
       [3.3901333e-06],
       [9.9206626e-01],
       [9.9999976e-01]], dtype=float32)

In [ ]:
pred_y2 = np.round(pred_y, decimals = 1)
pred_y2=pred_y2.astype('int64')

And the **confusion** **matrix**

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_y, pred_y2)


array([[1989,   57],
       [ 278, 1836]])

Finally, **false** **positives** and **false** **negatives** are stored in two csv files

In [ ]:
def getFP_FN_lists(test_X, test_y, pred_y):
    FP_text = []
    FP_index = []
    FN_text = []
    FN_index = []
    for i in range(len(test_y)):
        if(pred_y2[i]==1 and test_y[test_y.index[i]]==0):
            FP_text.append(test['text'][test_y.index[i]])
            FP_index.append(test_y.index[i])
        elif(pred_y2[i]==0 and test_y[test_y.index[i]]==1):
            FN_text.append(test['text'][test_y.index[i]])
            FN_index.append(test_y.index[i])
            
    return FP_text,FP_index,FN_text,FN_index

In [ ]:
def getFP_FN(test_X, test_y, pred_y2):
    FP_text,FP_index,FN_text,FN_index = getFP_FN_lists(test_X, test_y, pred_y2)
    d_FP = {'FP_text':FP_text,'FP_index':FP_index}
    df_FP = pd.DataFrame(d_FP)
    d_FN = {'FN_text':FN_text,'FN_index':FN_index}
    df_FN = pd.DataFrame(d_FN)
   
    return df_FP,df_FN

In [ ]:
df_FP,df_FN = getFP_FN(test_X, test_y, pred_y2)
df_FP.to_csv('FP_CNN2.csv', index=True)
df_FN.to_csv('FN_CNN2.csv', index=True)